<a href="https://colab.research.google.com/github/ccloveak/Study_Notes/blob/master/%E4%B8%80%E5%B0%8F%E6%97%B6%E5%85%A5%E9%97%A8_Python_3_%E7%BD%91%E7%BB%9C%E7%88%AC%E8%99%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#网络爬虫简介

网络爬虫，也叫网络蜘蛛(Web Spider)，根据网页地址(URL)爬取网页内。

###审查元素  
浏览器就是作为客户端从服务器端获取信息，然后将信息解析，并展示。可以在本地修改HTML信息，修改的信息不会回传到服务器，服务器存储的HTML信息不会改变。

###  简单实例

*   urllib.request
*   requests  [github](https://github.com/requests/requests)



##### requests安装

In [1]:
!pip install requests

##### 简单实例  
![基础方法](http://images.gitbook.cn/b30af580-9705-11e7-9b63-61d45538243e)  
[官方中文教程](http://docs.python-requests.org/zh_CN/latest/user/quickstart.html)

In [0]:
import requests
if __name__ == '__main__':
    target = 'http://gitbook.cn/'
    req = requests.get(url=target)
    print(req.text)

*   requests.get()方法必须设置的一个参数就是url，因为我们得告诉GET请求，我们的目标是谁，我们要获取谁的信息
*   将GET请求获得的响应内容存放到req变量中，然后使用req.text就可以获得HTML信息


### 爬虫实战




1.   用requests获取整个网页的HTML信息
2.   解析HTML信息，提取感兴趣的内容 [Beautiful Soup](http://beautifulsoup.readthedocs.io/zh_CN/latest/)



```
    import requests

    if __name__ == '__main__':
        target = 'http://www.biqukan.com/1_1094/5403177.html'
        req = requests.get(url=target)
        print(req.text)
```



***
```
    from bs4 import BeautifulSoup
    import requests

    if __name__ == "__main__":
        target = 'http://www.biqukan.com/1_1094/5403177.html'
        req = requests.get(url = target)
        html = req.text
        bf = BeautifulSoup(html)
        texts = bf.find_all('div', class_ = 'showtxt')
        print(texts)
```

在解析html之前，我们需要创建一个Beautiful Soup对象。BeautifulSoup函数里的参数就是我们已经获得的html信息。然后我们使用find_all方法，获得html信息中所有class属性为showtxt的div标签。  

find_all方法的第一个参数是获取的标签名，第二个参数class_是标签的属性，为什么不是class，而带了一个下划线呢？因为python中class是关键字，为了防止冲突，这里使用class_表示标签的class属性，class_后面跟着的showtxt就是属性值了。

```
<div id="content", class="showtxt">     

```

3.  去除div标签名，br标签，以及各种空格




```
from bs4 import BeautifulSoup
    import requests

    if __name__ == "__main__":
        target = 'http://www.biqukan.com/1_1094/5403177.html'
        req = requests.get(url = target)
        html = req.text
        bf = BeautifulSoup(html)
        texts = bf.find_all('div', class_ = 'showtxt')
        print(texts[0].text.replace('\xa0'*8,'\n\n'))
```

find_all匹配的返回的结果是一个列表。  
提取匹配结果后，使用text属性，提取文本内容，滤除br标签。  
随后使用replace方法，剔除空格，替换为回车进行分段。  
&nbsp;在html中是用来表示空格的。  
`replace('\xa0'*8,'\n\n')`就是去掉下图的八个空格符号，并用回车代替


**分析下小说目录**  
小说每章的链接放在了class属性为listmain的`<div>`标签下的`<a>`标签中。链接具体位置放在`html->body->div->dl->dd->a`的href属性中  
先匹配class属性为`listmain的<div>`标签，再匹配`<a>`标签。



```
    from bs4 import BeautifulSoup
    import requests

    if __name__ == "__main__":
        target = 'http://www.biqukan.com/1_1094/'
        req = requests.get(url = target)
        html = req.text
        div_bf = BeautifulSoup(html)
        div = div_bf.find_all('div', class_ = 'listmain')
        print(div[0])
```



匹配每一个`<a>`标签，并提取章节名和章节文章  


*   对Beautiful Soup返回的匹配结果a，使用a.get('href')方法就能获取href的属性值
*   使用a.string就能获取章节名



```
from bs4 import BeautifulSoup
    import requests

    if __name__ == "__main__":
        server = 'http://www.biqukan.com/'
        target = 'http://www.biqukan.com/1_1094/'
        req = requests.get(url = target)
        html = req.text
        div_bf = BeautifulSoup(html)
        div = div_bf.find_all('div', class_ = 'listmain')
        a_bf = BeautifulSoup(str(div[0]))
        a = a_bf.find_all('a')
        for each in a:
            print(each.string, server + each.get('href'))
```
为find_all返回的是一个列表，里边存放了很多的`<a>`标签，所以使用for循环遍历每个`<a>`标签并打印出来


##### 整合代码  



```
from bs4 import BeautifulSoup
    import requests, sys

    """
    类说明:下载《笔趣看》网小说《一念永恒》
    Parameters:
        无
    Returns:
        无
    Modify:
        2017-09-13
    """
    class downloader(object):

        def __init__(self):
            self.server = 'http://www.biqukan.com/'
            self.target = 'http://www.biqukan.com/1_1094/'
            self.names = []         #存放章节名
            self.urls = []          #存放章节链接
            self.nums = 0           #章节数

        """
        函数说明:获取下载链接
        Parameters:
            无
        Returns:
            无
        Modify:
            2017-09-13
        """
        def get_download_url(self):
            req = requests.get(url = self.target)
            html = req.text
            div_bf = BeautifulSoup(html)
            div = div_bf.find_all('div', class_ = 'listmain')
            a_bf = BeautifulSoup(str(div[0]))
            a = a_bf.find_all('a')
            self.nums = len(a[15:])                             #剔除不必要的章节，并统计章节数
            for each in a[15:]:
                self.names.append(each.string)
                self.urls.append(self.server + each.get('href'))

        """
        函数说明:获取章节内容
        Parameters:
            target - 下载连接(string)
        Returns:
            texts - 章节内容(string)
        Modify:
            2017-09-13
        """
        def get_contents(self, target):
            req = requests.get(url = target)
            html = req.text
            bf = BeautifulSoup(html)
            texts = bf.find_all('div', class_ = 'showtxt')
            texts = texts[0].text.replace('\xa0'*8,'\n\n')
            return texts

        """
        函数说明:将爬取的文章内容写入文件
        Parameters:
            name - 章节名称(string)
            path - 当前路径下,小说保存名称(string)
            text - 章节内容(string)
        Returns:
            无
        Modify:
            2017-09-13
        """
        def writer(self, name, path, text):
            write_flag = True
            with open(path, 'a', encoding='utf-8') as f:
                f.write(name + '\n')
                f.writelines(text)
                f.write('\n\n')

    if __name__ == "__main__":
        dl = downloader()
        dl.get_download_url()
        print('《一年永恒》开始下载：')
        for i in range(dl.nums):
            dl.writer(dl.names[i], '一念永恒.txt', dl.get_contents(dl.urls[i]))
            sys.stdout.write("  已下载:%.3f%%" %  float(i/dl.nums) + '\r')
            sys.stdout.flush()
        print('《一年永恒》下载完成')
```



### 优美壁纸下载

URL：https://unsplash.com/



1.   使用requeusts获取整个网页的HTML信息；

2.    使用Beautiful Soup解析HTML信息，找到所有`<img>`标签，提取src属性，获取图片存放地址；


3.   根据图片存放地址，下载图片。



使用抓包工具  [Fiddler](ttp://www.telerik.com/fiddler)
使用方法：打开软件，然后用浏览器打开目标网站

编程提取json数据  


1.   获取整个json数据
2.   分析json数据



反爬虫手段，验证Request Headers

Accept、Accept-Encoding、Accept-Language、DPR、User-Agent、Viewport-Width、accept-version、Referer、x-unsplash-client、authorization、Connection、Host  


*   User-Agent：这里面存放浏览器的信息。可以看到上图的参数值，它表示我是通过Windows的Chrome浏览器，访问的这个服务器。如果我们不设置这个参数，用Python程序直接发送GET请求，服务器接受到的User-Agent信息就会是一个包含python字样的User-Agent。如果后台设计者验证这个User-Agent参数是否合法，不让带Python字样的User-Agent访问，这样就起到了反爬虫的作用。这是一个最简单的，最常用的反爬虫手段。


*   Referer：这个参数也可以用于反爬虫，它表示这个请求是从哪发出的。可以看到我们通过浏览器访问网站，这个请求是从`https://unsplash.com/`，这个地址发出的。如果后台设计者，验证这个参数，对于不是从这个地址跳转过来的请求一律禁止访问，这样就也起到了反爬虫的作用。



*   authorization：这个参数是基于AAA模型中的身份验证信息允许访问一种资源的行为。在我们用浏览器访问的时候，服务器会为访问者分配这个用户ID。如果后台设计者，验证这个参数，对于没有用户ID的请求一律禁止访问，这样就又起到了反爬虫的作用。



Unsplash是根据authorization反爬虫  

通过程序手动添加这个参数，然后再发送GET请求，就可以顺利访问了

```
    import requests

    if __name__ == '__main__':
        target = 'http://unsplash.com/napi/feeds/home'
        headers = {'authorization':'your Client-ID'}
        req = requests.get(url=target, headers=headers, verify=False)
        print(req.text)
```

**以上为了获得json数据**




使用json.load()方法解析数据   获得 图片的ID已经

通过字符串处理一下，就生成了我们需要的图片下载请求地址

**整合代码**  


```
import requests, json, time, sys
    from contextlib import closing

    class get_photos(object):

        def __init__(self):
            self.photos_id = []
            self.download_server = 'https://unsplash.com/photos/xxx/download?force=trues'
            self.target = 'http://unsplash.com/napi/feeds/home'
            self.headers = {'authorization':'your Client-ID'}

        """
        函数说明:获取图片ID
        Parameters:
            无
        Returns:
            无
        Modify:
            2017-09-13
        """ 
        def get_ids(self):
            req = requests.get(url=self.target, headers=self.headers, verify=False)
            html = json.loads(req.text)
            next_page = html['next_page']
            for each in html['photos']:
                self.photos_id.append(each['id'])
            time.sleep(1)
            for i in range(4):
                req = requests.get(url=next_page, headers=self.headers, verify=False)
                html = json.loads(req.text)
                next_page = html['next_page']
                for each in html['photos']:
                    self.photos_id.append(each['id'])
                time.sleep(1)


        """
        函数说明:图片下载
        Parameters:
            无
        Returns:
            无
        Modify:
            2017-09-13
        """ 
        def download(self, photo_id, filename):
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36'}
            target = self.download_server.replace('xxx', photo_id)
            with closing(requests.get(url=target, stream=True, verify = False, headers = self.headers)) as r:
                with open('%d.jpg' % filename, 'ab+') as f:
                    for chunk in r.iter_content(chunk_size = 1024):
                        if chunk:
                            f.write(chunk)
                            f.flush()

    if __name__ == '__main__':
        gp = get_photos()
        print('获取图片连接中:')
        gp.get_ids()
        print('图片下载中:')
        for i in range(len(gp.photos_id)):
            print('  正在下载第%d张图片' % (i+1))
            gp.download(gp.photos_id[i], (i+1))
```



### 爱奇艺VIP视频下载

通过旋风视频VIP解析网站  URL：http://api.xfsub.com/  
通用解析方式是： ` http://api.xfsub.com/index.php?url=[播放地址或视频id]`

使用Fiddler进行抓包


1.   分析包内容
2.   在抓包结果中，测试请求
3.   分析json数据
4.   得到视频格式的地址

url有时效性 反爬虫策略也有时效性



编程思路


1.   正则表达式匹配到key、time、url等信息。
2.   根据匹配的到信息发POST请求，获得一个存放视频信息的url。
3.   根据这个url获得视频存放的地址。
4.   根据最终的视频地址，下载视频。


编写代码
需要使用requests.session()保持我们的会话请求


```
import requests,re, json
    from bs4 import BeautifulSoup

    class video_downloader():
        def __init__(self, url):
            self.server = 'http://api.xfsub.com'
            self.api = 'http://api.xfsub.com/xfsub_api/?url='
            self.get_url_api = 'http://api.xfsub.com/xfsub_api/url.php'
            self.url = url.split('#')[0]
            self.target = self.api + self.url
            self.s = requests.session()

        """
        函数说明:获取key、time、url等参数
        Parameters:
            无
        Returns:
            无
        Modify:
            2017-09-18
        """
        def get_key(self):
            req = self.s.get(url=self.target)
            req.encoding = 'utf-8'
            self.info = json.loads(re.findall('"url.php",\ (.+),', req.text)[0])    #使用正则表达式匹配结果，将匹配的结果存入info变量中

        """
        函数说明:获取视频地址
        Parameters:
            无
        Returns:
            video_url - 视频存放地址
        Modify:
            2017-09-18
        """
        def get_url(self):
            data = {'time':self.info['time'],
                'key':self.info['key'],
                'url':self.info['url'],
                'type':''}
            req = self.s.post(url=self.get_url_api,data=data)
            url = self.server + json.loads(req.text)['url']
            req = self.s.get(url)
            bf = BeautifulSoup(req.text,'xml')                                      #因为文件是xml格式的，所以要进行xml解析。
            video_url = bf.find('file').string                                      #匹配到视频地址
                return video_url


    if __name__ == '__main__':
        url = 'http://www.iqiyi.com/v_19rr7qhfg0.html#vfrm=19-9-0-1'
        vd = video_downloader(url)
        vd.get_key()
        print(vd.get_url())
```




根据视频地址，使用urllib.request.urlretrieve()即可将视频下载下来


```
import requests,re, json, sys
    from bs4 import BeautifulSoup
    from urllib import request

    class video_downloader():
        def __init__(self, url):
            self.server = 'http://api.xfsub.com'
            self.api = 'http://api.xfsub.com/xfsub_api/?url='
            self.get_url_api = 'http://api.xfsub.com/xfsub_api/url.php'
            self.url = url.split('#')[0]
            self.target = self.api + self.url
            self.s = requests.session()

        """
        函数说明:获取key、time、url等参数
        Parameters:
            无
        Returns:
            无
        Modify:
            2017-09-18
        """
        def get_key(self):
            req = self.s.get(url=self.target)
            req.encoding = 'utf-8'
            self.info = json.loads(re.findall('"url.php",\ (.+),', req.text)[0])    #使用正则表达式匹配结果，将匹配的结果存入info变量中

        """
        函数说明:获取视频地址
        Parameters:
            无
        Returns:
            video_url - 视频存放地址
        Modify:
            2017-09-18
        """
        def get_url(self):
            data = {'time':self.info['time'],
                'key':self.info['key'],
                'url':self.info['url'],
                'type':''}
            req = self.s.post(url=self.get_url_api,data=data)
            url = self.server + json.loads(req.text)['url']
            req = self.s.get(url)
            bf = BeautifulSoup(req.text,'xml')                                      #因为文件是xml格式的，所以要进行xml解析。
            video_url = bf.find('file').string                                      #匹配到视频地址
            return video_url

        """
        函数说明:回调函数，打印下载进度
        Parameters:
            a b c - 返回信息
        Returns:
            无
        Modify:
            2017-09-18
        """
        def Schedule(self, a, b, c):
            per = 100.0*a*b/c
            if per > 100 :
                per = 1
            sys.stdout.write("  " + "%.2f%% 已经下载的大小:%ld 文件大小:%ld" % (per,a*b,c) + '\r')
            sys.stdout.flush()

        """
        函数说明:视频下载
        Parameters:
            url - 视频地址
            filename - 视频名字
        Returns:
            无
        Modify:
            2017-09-18
        """
        def video_download(self, url, filename):
            request.urlretrieve(url=url,filename=filename,reporthook=self.Schedule)

    if __name__ == '__main__':
        url = 'http://www.iqiyi.com/v_19rr7qhfg0.html#vfrm=19-9-0-1'
        vd = video_downloader(url)
        filename = '加勒比海盗5'
        print('%s下载中:' % filename)
        vd.get_key()
        video_url = vd.get_url()
        print('  获取地址成功:%s' % video_url)
        vd.video_download(video_url, filename+'.mp4')
        print('\n下载完成！')
```



urlretrieve()有三个参数

1.   第一个url参数是视频存放的地址
2.   第二个参数filename是保存的文件名
3.   回调函数，它方便我们查看下载进度



###参考 
[Github](https://github.com/Jack-Cherish/python-spider)